In [1]:
import pandas as pd
import numpy as np
import random

#split dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#AUCROC,AUCPR,precision,recall,f1-score
from sklearn.metrics import roc_curve,auc,average_precision_score,precision_score,recall_score,f1_score

#gridsearch/randomsearch
from itertools import product
from tqdm import tqdm

#visualize results
import matplotlib.pyplot as plt
import time

#Isolation Forest
from sklearn.ensemble import IsolationForest

In [2]:
#remove randomness
def set_seed(seed):
  np.random.seed(seed)
  random.seed(seed)

In [3]:
def data_prepare(contam_ratio,seed):
    set_seed(seed)
    data = pd.read_csv("E:/Research_Anomaly Detection/Taiwanese Bankruptcy Prediction (TBP)/data/TBP.csv")
    #change 0,1 label to 1,-1
    data.loc[data['Flag'] == 1,'Flag'] = -1
    data.loc[data['Flag'] == 0,'Flag'] = 1

    #X = np.array(data.drop(['Flag'], axis = 1))
    X = np.array(data.iloc[:,1:20])
    y = np.array(data['Flag'].values)
    
    #reverse
    scaler = MinMaxScaler().fit(X)
    X = scaler.inverse_transform(X)

    #split the data to training, validation and testing data (50%,20%,30%)
#     X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,shuffle = False)
#     X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 2/7,shuffle = False)

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,shuffle = True,stratify = y)
    X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 2/7,shuffle = True,stratify = y_train)

    #the known positive samples before contaminating
    known_pos_entire = sum(y_train == -1)
    #Minmax
    #reverse
    scaler=MinMaxScaler().fit(X_train)

    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    index_contam = np.arange(len(y_train))[y_train == -1]
    index_contam = np.random.choice(index_contam,int(contam_ratio*len(index_contam)),replace = False)

    y_train[index_contam] = 1
    #the known positive samples after contaminating
    known_pos_sub = sum(y_train == -1)

    print(f'The left true(known) positive samples in the training set:{known_pos_sub}/{known_pos_entire}\n')
    
    return X_train,y_train,X_val,y_val,X_test,y_test

Isolation Forest

In [4]:
seed_pool = [1,2,3,4,5]
anomaly_ratio_pool = [0.03,0.04,0.05]
contam_ratio_pool = [1.0,0.98,0.8,0.5,0.0]

#random search size
search_size = 10

In [5]:
hyper_n_estimators = [50,100,150]
hyper_max_features = [0.6,0.8,1.0]

hyper_list_entire = list(product(hyper_n_estimators,hyper_max_features))

def random_search(hyper_list_entire, search_size, seed):
    if search_size < len(hyper_list_entire):
        set_seed(seed)
        index = np.random.choice(np.arange(len(hyper_list_entire)), search_size, replace=False)

        hyper_list = []
        for i in index:
            hyper_list.append(hyper_list_entire[i])
    else:
        hyper_list = hyper_list_entire

    return hyper_list_entire

In [6]:
for contam_ratio in tqdm(contam_ratio_pool):
  df_result = pd.DataFrame(data = None,index = ['AUCPR'] + anomaly_ratio_pool,columns = seed_pool)
  for seed in tqdm(seed_pool):
    #############################################seleting the best hyper-parameters in validation set#############################################
    metric_value_list=list()
    hyper_list = random_search(hyper_list_entire,search_size,seed)
    for i in range(len(hyper_list)):
      #try:
        print(f'Finding Optimal Hyper-parameters......Current Candidates: {hyper_list[i]}')
        n_estimators,max_features = hyper_list[i]
        #data
        X_train,_,X_val,y_val,_,_ = data_prepare(contam_ratio,seed)
        #model initialization
        model_if = IsolationForest(n_estimators = n_estimators,max_features = max_features,random_state = seed)
        #fitting
        model_if.fit(X_train)
        #evaluation
        score = -model_if.decision_function(X_val)
        metric_value = average_precision_score(y_true = y_val,y_score = score,pos_label = -1)
        metric_value_list.append(metric_value)

        print(f'The metric value corresponded to the hyper-parameters is :{metric_value:{.4}}')
        print('******************************')
        print('\n')
      #except:
        #pass
      #continue

    best_hyper_params=hyper_list[metric_value_list.index(max(metric_value_list))]
    print(f'The best hyper-parameters are: {best_hyper_params}')
    print('\n')
    ###################################################################testing#########################################################################
    print('Testing Phrase......')
    n_estimators,max_features = best_hyper_params

    #data
    X_train,_,_,_,X_test,y_test = data_prepare(contam_ratio,seed)
    
    #model initialization
    model_if=IsolationForest(n_estimators = n_estimators,max_features = max_features,random_state = seed)
    #fitting
    model_if.fit(X_train)
    #evaluation
    score = -model_if.decision_function(X_test)
    
    #store the result
    #AUCPR
    df_result.loc['AUCPR',seed] = average_precision_score(y_true = y_test,y_score = score,pos_label = -1)
    #F1
    for anomaly_ratio in anomaly_ratio_pool:
        threshold = score[np.argsort(-score)][int(anomaly_ratio*len(score))]
        
        y_pred = np.ones(len(score))
        y_pred[score >= threshold] = -1
        
        print('\n')
        print(f'Precision: {round(precision_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
        print(f'Recall: {round(recall_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
        print(f'F1-score: {round(f1_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
        print('\n')

        df_result.loc[anomaly_ratio,seed] = f1_score(y_pred = y_pred,y_true = y_test,pos_label = -1) 

  #mean & sd
  df_result['mean'] = np.mean(df_result.loc[:,seed_pool],axis = 1)
  df_result['std'] = np.std(df_result.loc[:,seed_pool],axis = 1)
  df_result = round(df_result.astype('float64')*100,2)

  filepath = 'E:/Research_Anomaly Detection/Taiwanese Bankruptcy Prediction (TBP)/result/' +\
              'TBP_IF_' + str(contam_ratio) + '.csv'
  df_result.to_csv(filepath,index = False)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Finding Optimal Hyper-parameters......Current Candidates: (50, 0.6)
The left true(known) positive samples in the training set:0/110

The metric value corresponded to the hyper-parameters is :0.101
******************************


Finding Optimal Hyper-parameters......Current Candidates: (50, 0.8)
The left true(known) positive samples in the training set:0/110

The metric value corresponded to the hyper-parameters is :0.1185
******************************


Finding Optimal Hyper-parameters......Current Candidates: (50, 1.0)
The left true(known) positive samples in the training set:0/110

The metric value corresponded to the hyper-parameters is :0.1095
******************************


Finding Optimal Hyper-parameters......Current Candidates: (100, 0.6)
The left true(known) positive samples in the training set:0/110

The metric value corresponded to the hyper-parameters is :0.1047
******************************


Finding Optimal Hyper-parameters......Current Candidates: (100, 0.8)
The lef


 20%|████████████████▊                                                                   | 1/5 [00:10<00:42, 10.55s/it]



Precision: 19.35
Recall: 18.18
F1-score: 18.75




Precision: 15.85
Recall: 19.7
F1-score: 17.57




Precision: 15.53
Recall: 24.24
F1-score: 18.93


Finding Optimal Hyper-parameters......Current Candidates: (50, 0.6)
The left true(known) positive samples in the training set:0/110

The metric value corresponded to the hyper-parameters is :0.1131
******************************


Finding Optimal Hyper-parameters......Current Candidates: (50, 0.8)
The left true(known) positive samples in the training set:0/110

The metric value corresponded to the hyper-parameters is :0.1346
******************************


Finding Optimal Hyper-parameters......Current Candidates: (50, 1.0)
The left true(known) positive samples in the training set:0/110

The metric value corresponded to the hyper-parameters is :0.09975
******************************


Finding Optimal Hyper-parameters......Current Candidates: (100, 0.6)
The left true(known) positive samples in the training set:0/110

The metric value cor


 40%|█████████████████████████████████▌                                                  | 2/5 [00:22<00:33, 11.10s/it]

...



Precision: 8.06
Recall: 7.58
F1-score: 7.81




Precision: 14.63
Recall: 18.18
F1-score: 16.22




Precision: 15.53
Recall: 24.24
F1-score: 18.93


Finding Optimal Hyper-parameters......Current Candidates: (50, 0.6)
The left true(known) positive samples in the training set:110/110

The metric value corresponded to the hyper-parameters is :0.06439
******************************


Finding Optimal Hyper-parameters......Current Candidates: (50, 0.8)
The left true(known) positive samples in the training set:110/110

The metric value corresponded to the hyper-parameters is :0.07237
******************************


Finding Optimal Hyper-parameters......Current Candidates: (50, 1.0)
The left true(known) positive samples in the training set:110/110

The metric value corresponded to the hyper-parameters is :0.0725
******************************


Finding Optimal Hyper-parameters......Current Candidates: (100, 0.6)
The left true(known) positive samples in the training set:110/110

The metric va


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:43<00:11, 11.07s/it]



Precision: 16.13
Recall: 15.15
F1-score: 15.62




Precision: 15.85
Recall: 19.7
F1-score: 17.57




Precision: 14.56
Recall: 22.73
F1-score: 17.75


Finding Optimal Hyper-parameters......Current Candidates: (50, 0.6)
The left true(known) positive samples in the training set:110/110

The metric value corresponded to the hyper-parameters is :0.06152
******************************


Finding Optimal Hyper-parameters......Current Candidates: (50, 0.8)
The left true(known) positive samples in the training set:110/110

The metric value corresponded to the hyper-parameters is :0.06704
******************************


Finding Optimal Hyper-parameters......Current Candidates: (50, 1.0)
The left true(known) positive samples in the training set:110/110

The metric value corresponded to the hyper-parameters is :0.06438
******************************


Finding Optimal Hyper-parameters......Current Candidates: (100, 0.6)
The left true(known) positive samples in the training set:110/110

The metric


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:58<00:00, 59.80s/it]



Precision: 12.9
Recall: 12.12
F1-score: 12.5




Precision: 12.2
Recall: 15.15
F1-score: 13.51




Precision: 13.59
Recall: 21.21
F1-score: 16.57


